In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import opttrot


## Fast Pauli decomposition algorithm implementation

```
Hantzko, Binkowski, and Gupta, Tensorized Pauli decomposition algorithm
```

In [3]:
import numpy as np
from copy import deepcopy

In [4]:
from opttrot import utils 

In [5]:
# Size = 2^n Hermit matrix
n =5
size_n = int(2**n)
A = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
ImA = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
A = A.H@A
ImA = ImA.H@ImA
H = A+1j*ImA

In [6]:
pauli_matrix, pauli_terms = opttrot.Hamiltonian.generate_pauli_terms(n) # Get 2^n pauli-terms

In [7]:
# Frobenius inner product
decomposition = {}
for matrix, term in zip(pauli_matrix, pauli_terms):
    coef = utils.frobenius_inner(matrix, H)
    decomposition[term] = coef

In [29]:
decomposition

{'IIIII': (10.853874257475193+10.184160259663344j),
 'IIIIZ': (-0.0381030463418246+0.21405928361658821j),
 'IIIZI': (0.5368206047579829+0.24889025149281974j),
 'IIIZZ': (-0.03809535773803896-0.04503156214692394j),
 'IIZII': (-0.1426307531241302+0.12973326517074668j),
 'IIZIZ': (-0.005790992883256463-0.33904506697236403j),
 'IIZZI': (0.02971424466314221+0.1127601323425107j),
 'IIZZZ': (0.30922674481246326+0.006018844330100359j),
 'IZIII': (0.12089225549234939+0.026554743304782524j),
 'IZIIZ': (-0.47319086383494213+0.5546828665916788j),
 'IZIZI': (0.26612076074466395-0.40541671931030915j),
 'IZIZZ': (0.19003478693631604+0.007838495959633873j),
 'IZZII': (0.11395937812401494-0.4688843049025599j),
 'IZZIZ': (-0.25585856317792577-0.2980617608398181j),
 'IZZZI': (-0.3765307636117148+0.42265775270872125j),
 'IZZZZ': (-0.38470600985186465+0.02152445305656775j),
 'ZIIII': (0.15236696828784663+0.06204915598316657j),
 'ZIIIZ': (-0.2989677687347954-0.07236420783525904j),
 'ZIIZI': (-0.280704054177

In [30]:
H_test = deepcopy(H)

### Implementation

In [31]:
exp2n = H_test.shape[0]
n = int(np.log2(exp2n))

print("Hamiltonian shape:", H_test.shape)
print("Total qubits:", n)


Hamiltonian shape: (32, 32)
Total qubits: 5


In [32]:
H_test = deepcopy(H)

In [33]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5


In [34]:
def tensorized_decomposition(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n)) # wires

    l = int(exp2n)

    for i in range(n): #range(n)
        m = int(2**i) # Number of submatrix
        l = int(l/2) # Sub matrix size, square
        for j in range(m):
            for k in range(m):
                num_i = j*(2*l) # Initial position of sub matrix row
                num_j = k*(2*l) # Initial position of sub matrix column
                #print(f"{num_i}, {num_j}")
                #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")

                H[num_i: num_i+l, num_j:num_j+l]        += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
        H *= 0.5
    return H

In [25]:
def index_xzcode(i, j):
    return i^j, i
def xzcode_index(nx, nz):
    return nz, nx^nz

In [26]:
opttrot.utils.pstr_from_xz(2, 4)

'ZXI'

In [35]:
H_test = deepcopy(H)

In [36]:
H_decom = tensorized_decomposition(H_test)
nr, nc = H_decom.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_decom[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [37]:
nr, nc = H_test.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_test[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [38]:
decom

{'IIIII': (10.853874257475194+10.18416025966334j),
 'IIIIX': (7.892014618157073+7.657922124497867j),
 'IIIXI': (8.062037559033488+7.55949298052043j),
 'IIIXX': (7.902859582364991+7.730898355936536j),
 'IIXII': (7.867070262426058+7.683251636867523j),
 'IIXIX': (8.106590032114381+7.5008486257949745j),
 'IIXXI': (8.01332807676594+7.619220360399859j),
 'IIXXX': (8.196178247788813+7.605418469492358j),
 'IXIII': (8.020483699960835+7.64050826601099j),
 'IXIIX': (7.984242138531815+7.50106324181062j),
 'IXIXI': (8.04302535066217+7.553439471959424j),
 'IXIXX': (8.0882908932759+7.625230717613096j),
 'IXXII': (8.04907799886482+7.610126748455368j),
 'IXXIX': (8.154022522851339+7.677058954147354j),
 'IXXXI': (8.21162286388413+7.597474807901156j),
 'IXXXX': (7.860586193419396+7.731187595942259j),
 'XIIII': (7.969007539688908+7.769305187988721j),
 'XIIIX': (8.199345549653643+7.639453319451109j),
 'XIIXI': (7.866866533806624+7.489187918798835j),
 'XIIXX': (8.151785416530759+7.5469650431163045j),
 'XIXI

In [17]:
diff = {}
for key in decom.keys():
    diff[key] = abs(decomposition[key] - decom[key])

In [18]:
np.array(list(diff.values())).max()

3.972054645195637e-15

In [19]:
from pd import PauliDecomposition

In [20]:
pstring, pcoef = PauliDecomposition(H)
pdecompose = {st:coef for st, coef in zip(pstring, pcoef)}

In [21]:
pdecompose

{'IIIII': (10.853874257475194+10.18416025966334j),
 'IIIIX': (7.892014618157073+7.657922124497867j),
 'IIIIZ': (-0.038103046341824154+0.21405928361658777j),
 'IIIXI': (8.062037559033488+7.55949298052043j),
 'IIIXX': (7.902859582364991+7.730898355936536j),
 'IIIXZ': (0.14857623324105607-0.07324290588027615j),
 'IIIYY': (3.073506216555444e-05-0.032758444643328666j),
 'IIIZI': (0.5368206047579824+0.2488902514928193j),
 'IIIZX': (0.3952252571589385+0.20053297998492292j),
 'IIIZZ': (-0.03809535773803896-0.04503156214692394j),
 'IIXII': (7.867070262426058+7.683251636867523j),
 'IIXIX': (8.106590032114381+7.5008486257949745j),
 'IIXIZ': (-0.10944873121256427+0.04161408109994813j),
 'IIXXI': (8.01332807676594+7.619220360399859j),
 'IIXXX': (8.196178247788813+7.605418469492358j),
 'IIXXZ': (0.11252180489895736-0.05017084481319145j),
 'IIXYY': (-0.09185819619650637-0.057069496277881715j),
 'IIXZI': (0.5026416952011734+0.3726100127171934j),
 'IIXZX': (0.33379822102604706+0.08612172046165911j),
 '

In [23]:
diff2 = {}
for key in decom.keys():
    try:
        diff2[key] = abs(pdecompose[key] - decom[key])
    except:
        print(decom[key])

0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
(4.440892098500626e-16+0j)
0j
(4.440892098500626e-16-8.881784197001252e-16j)
0j
(-4.440892098500626e-16+0j)
0j
0j
0j
(-4.440892098500626e-16+0j)
0j
0j
0j
(4.440892098500626e-16+0j)
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
(-2.220446049250313e-16+0j)
4.440892098500626e-16j
(-2.220446049250313e-16-6.661338147750939e-16j)
0j
2.220446049250313e-16j
2.220446049250313e-16j
(-2.220446049250313e-16+0j)
0j
-2.220446049250313e-16j
0j
-2.220446049250313e-16j
0j
(-2.220446049250313e-16+2.220446049250313e-16j)
(-2.220446049250313e-16-4.440892098500626e-16j)
4.440892098500626e-16j
0j
4.440892098500626e-16j
0j
0j
0j
(-4.440892098500626e-16+2.220446049250313e-16j)
0j
0j
0j
(-4.440892098500626e-16+8.881784197001252e-16j)
0j
0j
0j
(2.220446049250313e-16-4.440892098500626e-16j)
0j
0j
0j
(2.220446049250313e-16+0j)
0j
0j
0j
-2.220446049250313e-16j
0j
0j
0j
2.220446049250313e-16j
0j
0j
0j
(-2.220446049250313e-16-2.220446049250313e-16j)
0j
0j

In [14]:
def fast_p_decompose(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n))
    # Implementation of decomposition
    l = int(exp2n)
    for i in range(n):
        m = int(2**i)
        l = int(l/2)
        for j in range(m):
            for k in range(m):

                num_i = j*l
                num_j = k*l

                H[num_i: num_i+l, num_j:num_j+l] += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j

        H *= 0.5
    return H

# string to matrix

In [175]:
from opttrot.utils import pstr_to_matrix, pstr_to_xz_fam_code

In [176]:
pstr= "IZX"
xz_code = pstr_to_xz_fam_code(pstr)
p_matrix = pstr_to_matrix(pstr)

In [177]:
p_matrix

array([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -1.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,
        -0.+0.j]])

In [179]:
xz_code

(1, 2)

In [182]:
mat = np.zeros(p_matrix.shape)
mat[*xzcode_index(*xz_code)] = 1

In [ ]:
# Single Pauli term to matrix

In [ ]:
# Multiple Pauli term at once
# Inverse routine of decomposition

In [ ]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5
